In [1]:
import polartoolkit
from polartoolkit import fetch, maps, profiles, regions

In [10]:
import pathlib
import pooch
from tqdm import tqdm
import requests
import pandas as pd

from utils import points_inside_region

ModuleNotFoundError: No module named 'utils'

In [3]:
import logging


logger = logging.getLogger(__name__)

In [4]:
import verde as vd
import invert4geom
import numpy as np
import pandas as pd
import harmonica
import matplotlib.pyplot as plt

In [5]:
%load_ext autoreload
%autoreload 2
from polartoolkit import fetch, maps, profiles, regions

In [6]:
import os

os.environ["POLARTOOLKIT_HEMISPHERE"] = "south"

region = regions.getz_ice_shelf

In [7]:
url_list = ['https://data.nsidc.earthdatacloud.nasa.gov/nsidc-cumulus-prod-protected/ICEBRIDGE/IGGRV1B/2/2009/10/16/IGGRV1B_20091016_11515650_V020.txt', 'https://data.nsidc.earthdatacloud.nasa.gov/nsidc-cumulus-prod-public/ICEBRIDGE/IGGRV1B/2/2009/10/16/IGGRV1B_20091016_11515650_V020.txt.xml', 'https://data.nsidc.earthdatacloud.nasa.gov/nsidc-cumulus-prod-protected/ICEBRIDGE/IGGRV1B/2/2009/10/18/IGGRV1B_20091018_11442050_V020.txt', 'https://data.nsidc.earthdatacloud.nasa.gov/nsidc-cumulus-prod-public/ICEBRIDGE/IGGRV1B/2/2009/10/18/IGGRV1B_20091018_11442050_V020.txt.xml', 'https://data.nsidc.earthdatacloud.nasa.gov/nsidc-cumulus-prod-protected/ICEBRIDGE/IGGRV1B/2/2009/10/21/IGGRV1B_20091021_12433200_V020.txt', 'https://data.nsidc.earthdatacloud.nasa.gov/nsidc-cumulus-prod-public/ICEBRIDGE/IGGRV1B/2/2009/10/21/IGGRV1B_20091021_12433200_V020.txt.xml', 'https://data.nsidc.earthdatacloud.nasa.gov/nsidc-cumulus-prod-protected/ICEBRIDGE/IGGRV1B/2/2009/10/28/IGGRV1B_20091028_11592500_V020.txt', 'https://data.nsidc.earthdatacloud.nasa.gov/nsidc-cumulus-prod-public/ICEBRIDGE/IGGRV1B/2/2009/10/28/IGGRV1B_20091028_11592500_V020.txt.xml', 'https://data.nsidc.earthdatacloud.nasa.gov/nsidc-cumulus-prod-protected/ICEBRIDGE/IGGRV1B/2/2009/11/02/IGGRV1B_20091102_13064950_V020.txt', 'https://data.nsidc.earthdatacloud.nasa.gov/nsidc-cumulus-prod-public/ICEBRIDGE/IGGRV1B/2/2009/11/02/IGGRV1B_20091102_13064950_V020.txt.xml', 'https://data.nsidc.earthdatacloud.nasa.gov/nsidc-cumulus-prod-protected/ICEBRIDGE/IGGRV1B/2/2009/11/18/IGGRV1B_20091118_11492450_V020.txt', 'https://data.nsidc.earthdatacloud.nasa.gov/nsidc-cumulus-prod-public/ICEBRIDGE/IGGRV1B/2/2009/11/18/IGGRV1B_20091118_11492450_V020.txt.xml', 'https://data.nsidc.earthdatacloud.nasa.gov/nsidc-cumulus-prod-protected/ICEBRIDGE/IGGRV1B/2/2010/10/30/IGGRV1B_20101030_11531750_V020.txt', 'https://data.nsidc.earthdatacloud.nasa.gov/nsidc-cumulus-prod-public/ICEBRIDGE/IGGRV1B/2/2010/10/30/IGGRV1B_20101030_11531750_V020.txt.xml', 'https://data.nsidc.earthdatacloud.nasa.gov/nsidc-cumulus-prod-protected/ICEBRIDGE/IGGRV1B/2/2010/11/05/IGGRV1B_20101105_12532350_V020.txt', 'https://data.nsidc.earthdatacloud.nasa.gov/nsidc-cumulus-prod-public/ICEBRIDGE/IGGRV1B/2/2010/11/05/IGGRV1B_20101105_12532350_V020.txt.xml', 'https://data.nsidc.earthdatacloud.nasa.gov/nsidc-cumulus-prod-protected/ICEBRIDGE/IGGRV1B/2/2010/11/06/IGGRV1B_20101106_12554650_V020.txt', 'https://data.nsidc.earthdatacloud.nasa.gov/nsidc-cumulus-prod-public/ICEBRIDGE/IGGRV1B/2/2010/11/06/IGGRV1B_20101106_12554650_V020.txt.xml', 'https://data.nsidc.earthdatacloud.nasa.gov/nsidc-cumulus-prod-protected/ICEBRIDGE/IGGRV1B/2/2010/11/19/IGGRV1B_20101119_17290550_V020.txt', 'https://data.nsidc.earthdatacloud.nasa.gov/nsidc-cumulus-prod-public/ICEBRIDGE/IGGRV1B/2/2010/11/19/IGGRV1B_20101119_17290550_V020.txt.xml', 'https://data.nsidc.earthdatacloud.nasa.gov/nsidc-cumulus-prod-protected/ICEBRIDGE/IGGRV1B/2/2011/10/17/IGGRV1B_20111017_11494300_V020.txt', 'https://data.nsidc.earthdatacloud.nasa.gov/nsidc-cumulus-prod-public/ICEBRIDGE/IGGRV1B/2/2011/10/17/IGGRV1B_20111017_11494300_V020.txt.xml', 'https://data.nsidc.earthdatacloud.nasa.gov/nsidc-cumulus-prod-protected/ICEBRIDGE/IGGRV1B/2/2011/10/23/IGGRV1B_20111023_11491000_V020.txt', 'https://data.nsidc.earthdatacloud.nasa.gov/nsidc-cumulus-prod-public/ICEBRIDGE/IGGRV1B/2/2011/10/23/IGGRV1B_20111023_11491000_V020.txt.xml', 'https://data.nsidc.earthdatacloud.nasa.gov/nsidc-cumulus-prod-protected/ICEBRIDGE/IGGRV1B/2/2011/11/03/IGGRV1B_20111103_11464750_V020.txt', 'https://data.nsidc.earthdatacloud.nasa.gov/nsidc-cumulus-prod-public/ICEBRIDGE/IGGRV1B/2/2011/11/03/IGGRV1B_20111103_11464750_V020.txt.xml', 'https://data.nsidc.earthdatacloud.nasa.gov/nsidc-cumulus-prod-protected/ICEBRIDGE/IGGRV1B/2/2011/11/04/IGGRV1B_20111104_14314700_V020.txt', 'https://data.nsidc.earthdatacloud.nasa.gov/nsidc-cumulus-prod-public/ICEBRIDGE/IGGRV1B/2/2011/11/04/IGGRV1B_20111104_14314700_V020.txt.xml', 'https://data.nsidc.earthdatacloud.nasa.gov/nsidc-cumulus-prod-protected/ICEBRIDGE/IGGRV1B/2/2011/11/09/IGGRV1B_20111109_12031600_V020.txt', 'https://data.nsidc.earthdatacloud.nasa.gov/nsidc-cumulus-prod-public/ICEBRIDGE/IGGRV1B/2/2011/11/09/IGGRV1B_20111109_12031600_V020.txt.xml', 'https://data.nsidc.earthdatacloud.nasa.gov/nsidc-cumulus-prod-protected/ICEBRIDGE/IGGRV1B/2/2011/11/11/IGGRV1B_20111111_13592250_V020.txt', 'https://data.nsidc.earthdatacloud.nasa.gov/nsidc-cumulus-prod-public/ICEBRIDGE/IGGRV1B/2/2011/11/11/IGGRV1B_20111111_13592250_V020.txt.xml', 'https://data.nsidc.earthdatacloud.nasa.gov/nsidc-cumulus-prod-protected/ICEBRIDGE/IGGRV1B/2/2011/11/12/IGGRV1B_20111112_13481200_V020.txt', 'https://data.nsidc.earthdatacloud.nasa.gov/nsidc-cumulus-prod-public/ICEBRIDGE/IGGRV1B/2/2011/11/12/IGGRV1B_20111112_13481200_V020.txt.xml', 'https://data.nsidc.earthdatacloud.nasa.gov/nsidc-cumulus-prod-protected/ICEBRIDGE/IGGRV1B/2/2011/11/13/IGGRV1B_20111113_12430500_V020.txt', 'https://data.nsidc.earthdatacloud.nasa.gov/nsidc-cumulus-prod-public/ICEBRIDGE/IGGRV1B/2/2011/11/13/IGGRV1B_20111113_12430500_V020.txt.xml', 'https://data.nsidc.earthdatacloud.nasa.gov/nsidc-cumulus-prod-protected/ICEBRIDGE/IGGRV1B/2/2012/10/12/IGGRV1B_20121012_11550450_V020.txt', 'https://data.nsidc.earthdatacloud.nasa.gov/nsidc-cumulus-prod-public/ICEBRIDGE/IGGRV1B/2/2012/10/12/IGGRV1B_20121012_11550450_V020.txt.xml', 'https://data.nsidc.earthdatacloud.nasa.gov/nsidc-cumulus-prod-protected/ICEBRIDGE/IGGRV1B/2/2012/10/13/IGGRV1B_20121013_13172000_V020.txt', 'https://data.nsidc.earthdatacloud.nasa.gov/nsidc-cumulus-prod-public/ICEBRIDGE/IGGRV1B/2/2012/10/13/IGGRV1B_20121013_13172000_V020.txt.xml', 'https://data.nsidc.earthdatacloud.nasa.gov/nsidc-cumulus-prod-protected/ICEBRIDGE/IGGRV1B/2/2012/10/23/IGGRV1B_20121023_13262900_V020.txt', 'https://data.nsidc.earthdatacloud.nasa.gov/nsidc-cumulus-prod-public/ICEBRIDGE/IGGRV1B/2/2012/10/23/IGGRV1B_20121023_13262900_V020.txt.xml', 'https://data.nsidc.earthdatacloud.nasa.gov/nsidc-cumulus-prod-protected/ICEBRIDGE/IGGRV1B/2/2012/10/27/IGGRV1B_20121027_11433100_V020.txt', 'https://data.nsidc.earthdatacloud.nasa.gov/nsidc-cumulus-prod-public/ICEBRIDGE/IGGRV1B/2/2012/10/27/IGGRV1B_20121027_11433100_V020.txt.xml', 'https://data.nsidc.earthdatacloud.nasa.gov/nsidc-cumulus-prod-protected/ICEBRIDGE/IGGRV1B/2/2013/11/25/IGGRV1B_20131125_18560050_V020.txt', 'https://data.nsidc.earthdatacloud.nasa.gov/nsidc-cumulus-prod-public/ICEBRIDGE/IGGRV1B/2/2013/11/25/IGGRV1B_20131125_18560050_V020.txt.xml', 'https://data.nsidc.earthdatacloud.nasa.gov/nsidc-cumulus-prod-protected/ICEBRIDGE/IGGRV1B/2/2014/11/02/IGGRV1B_20141102_11423800_V020.txt', 'https://data.nsidc.earthdatacloud.nasa.gov/nsidc-cumulus-prod-public/ICEBRIDGE/IGGRV1B/2/2014/11/02/IGGRV1B_20141102_11423800_V020.txt.xml', 'https://data.nsidc.earthdatacloud.nasa.gov/nsidc-cumulus-prod-protected/ICEBRIDGE/IGGRV1B/2/2014/11/03/IGGRV1B_20141103_11431000_V020.txt', 'https://data.nsidc.earthdatacloud.nasa.gov/nsidc-cumulus-prod-public/ICEBRIDGE/IGGRV1B/2/2014/11/03/IGGRV1B_20141103_11431000_V020.txt.xml', 'https://data.nsidc.earthdatacloud.nasa.gov/nsidc-cumulus-prod-protected/ICEBRIDGE/IGGRV1B/2/2014/11/07/IGGRV1B_20141107_11393500_V020.txt', 'https://data.nsidc.earthdatacloud.nasa.gov/nsidc-cumulus-prod-public/ICEBRIDGE/IGGRV1B/2/2014/11/07/IGGRV1B_20141107_11393500_V020.txt.xml', 'https://data.nsidc.earthdatacloud.nasa.gov/nsidc-cumulus-prod-protected/ICEBRIDGE/IGGRV1B/2/2014/11/11/IGGRV1B_20141111_11455100_V020.txt', 'https://data.nsidc.earthdatacloud.nasa.gov/nsidc-cumulus-prod-public/ICEBRIDGE/IGGRV1B/2/2014/11/11/IGGRV1B_20141111_11455100_V020.txt.xml', 'https://data.nsidc.earthdatacloud.nasa.gov/nsidc-cumulus-prod-protected/ICEBRIDGE/IGGRV1B/2/2014/11/13/IGGRV1B_20141113_11463900_V020.txt', 'https://data.nsidc.earthdatacloud.nasa.gov/nsidc-cumulus-prod-public/ICEBRIDGE/IGGRV1B/2/2014/11/13/IGGRV1B_20141113_11463900_V020.txt.xml', 'https://data.nsidc.earthdatacloud.nasa.gov/nsidc-cumulus-prod-protected/ICEBRIDGE/IGGRV1B/2/2014/11/21/IGGRV1B_20141121_11454150_V020.txt', 'https://data.nsidc.earthdatacloud.nasa.gov/nsidc-cumulus-prod-public/ICEBRIDGE/IGGRV1B/2/2014/11/21/IGGRV1B_20141121_11454150_V020.txt.xml', 'https://data.nsidc.earthdatacloud.nasa.gov/nsidc-cumulus-prod-protected/ICEBRIDGE/IGGRV1B/2/2016/10/20/IGGRV1B_20161020_12504700_V020.txt', 'https://data.nsidc.earthdatacloud.nasa.gov/nsidc-cumulus-prod-public/ICEBRIDGE/IGGRV1B/2/2016/10/20/IGGRV1B_20161020_12504700_V020.txt.xml', 'https://data.nsidc.earthdatacloud.nasa.gov/nsidc-cumulus-prod-protected/ICEBRIDGE/IGGRV1B/2/2016/10/26/IGGRV1B_20161026_12495550_V020.txt', 'https://data.nsidc.earthdatacloud.nasa.gov/nsidc-cumulus-prod-public/ICEBRIDGE/IGGRV1B/2/2016/10/26/IGGRV1B_20161026_12495550_V020.txt.xml', 'https://data.nsidc.earthdatacloud.nasa.gov/nsidc-cumulus-prod-protected/ICEBRIDGE/IGGRV1B/2/2016/10/28/IGGRV1B_20161028_12535500_V020.txt', 'https://data.nsidc.earthdatacloud.nasa.gov/nsidc-cumulus-prod-public/ICEBRIDGE/IGGRV1B/2/2016/10/28/IGGRV1B_20161028_12535500_V020.txt.xml', 'https://data.nsidc.earthdatacloud.nasa.gov/nsidc-cumulus-prod-protected/ICEBRIDGE/IGGRV1B/2/2016/10/31/IGGRV1B_20161031_13030750_V020.txt', 'https://data.nsidc.earthdatacloud.nasa.gov/nsidc-cumulus-prod-public/ICEBRIDGE/IGGRV1B/2/2016/10/31/IGGRV1B_20161031_13030750_V020.txt.xml', 'https://data.nsidc.earthdatacloud.nasa.gov/nsidc-cumulus-prod-protected/ICEBRIDGE/IGGRV1B/2/2016/11/02/IGGRV1B_20161102_12471700_V020.txt', 'https://data.nsidc.earthdatacloud.nasa.gov/nsidc-cumulus-prod-public/ICEBRIDGE/IGGRV1B/2/2016/11/02/IGGRV1B_20161102_12471700_V020.txt.xml', 'https://data.nsidc.earthdatacloud.nasa.gov/nsidc-cumulus-prod-protected/ICEBRIDGE/IGGRV1B/2/2016/11/03/IGGRV1B_20161103_13115450_V020.txt', 'https://data.nsidc.earthdatacloud.nasa.gov/nsidc-cumulus-prod-public/ICEBRIDGE/IGGRV1B/2/2016/11/03/IGGRV1B_20161103_13115450_V020.txt.xml', 'https://data.nsidc.earthdatacloud.nasa.gov/nsidc-cumulus-prod-protected/ICEBRIDGE/IGGRV1B/2/2016/11/05/IGGRV1B_20161105_12472350_V020.txt', 'https://data.nsidc.earthdatacloud.nasa.gov/nsidc-cumulus-prod-public/ICEBRIDGE/IGGRV1B/2/2016/11/05/IGGRV1B_20161105_12472350_V020.txt.xml', 'https://data.nsidc.earthdatacloud.nasa.gov/nsidc-cumulus-prod-protected/ICEBRIDGE/IGGRV1B/2/2016/11/07/IGGRV1B_20161107_12445850_V020.txt', 'https://data.nsidc.earthdatacloud.nasa.gov/nsidc-cumulus-prod-public/ICEBRIDGE/IGGRV1B/2/2016/11/07/IGGRV1B_20161107_12445850_V020.txt.xml', 'https://data.nsidc.earthdatacloud.nasa.gov/nsidc-cumulus-prod-protected/ICEBRIDGE/IGGRV1B/2/2016/11/09/IGGRV1B_20161109_12445200_V020.txt', 'https://data.nsidc.earthdatacloud.nasa.gov/nsidc-cumulus-prod-public/ICEBRIDGE/IGGRV1B/2/2016/11/09/IGGRV1B_20161109_12445200_V020.txt.xml', 'https://data.nsidc.earthdatacloud.nasa.gov/nsidc-cumulus-prod-protected/ICEBRIDGE/IGGRV1B/2/2016/11/15/IGGRV1B_20161115_12473400_V020.txt', 'https://data.nsidc.earthdatacloud.nasa.gov/nsidc-cumulus-prod-public/ICEBRIDGE/IGGRV1B/2/2016/11/15/IGGRV1B_20161115_12473400_V020.txt.xml', 'https://data.nsidc.earthdatacloud.nasa.gov/nsidc-cumulus-prod-protected/ICEBRIDGE/IGGRV1B/2/2018/10/31/IGGRV1B_20181031_14545400_V020.txt', 'https://data.nsidc.earthdatacloud.nasa.gov/nsidc-cumulus-prod-public/ICEBRIDGE/IGGRV1B/2/2018/10/31/IGGRV1B_20181031_14545400_V020.txt.xml', 'https://data.nsidc.earthdatacloud.nasa.gov/nsidc-cumulus-prod-protected/ICEBRIDGE/IGGRV1B/2/2018/11/11/IGGRV1B_20181111_12491100_V020.txt', 'https://data.nsidc.earthdatacloud.nasa.gov/nsidc-cumulus-prod-public/ICEBRIDGE/IGGRV1B/2/2018/11/11/IGGRV1B_20181111_12491100_V020.txt.xml', 'https://data.nsidc.earthdatacloud.nasa.gov/nsidc-cumulus-prod-protected/ICEBRIDGE/IGGRV1B/2/2018/11/15/IGGRV1B_20181115_12395800_V020.txt', 'https://data.nsidc.earthdatacloud.nasa.gov/nsidc-cumulus-prod-public/ICEBRIDGE/IGGRV1B/2/2018/11/15/IGGRV1B_20181115_12395800_V020.txt.xml']

In [8]:
def fetch_oib_gravity():
    """
    Fetch OIB gravity data from the NSIDC and merge into 1 file.

    """
    count=0

    if pathlib.Path(f"{pooch.os_cache('pooch')}/goat/gravity/oib_getz.csv").exists():
        df = pd.read_csv(
            f"{pooch.os_cache('pooch')}/goat/gravity/oib_getz.csv",
            # index=False,
        )
    else:
        urls = [t for t in url_list if t.endswith("txt")]

        dfs = []
        for url in tqdm(urls, total=len(urls), desc="txt files"):
            try:
                fname = pooch.retrieve(
                    url=url,
                    path=f"{pooch.os_cache('pooch')}/goat/gravity",
                    # fname="bedmap2_point_data.zip",
                    known_hash=None,
                    progressbar=True,
                    # processor=preprocessing,
                    downloader=fetch.EarthDataDownloader(),
                )
            except requests.exceptions.HTTPError as e:
                logger.error(e)
                logger.error(f"Failed to download {url}")
                count+=1
                continue
            df = pd.read_csv(
                fname,
                comment="#",
                header=None,
                usecols=[5,6,7,8,14,15,16,17],
                names=[
                    'line',
                    'x',
                    'y',
                    'height',
                    "levelling_correction",
                    'free_air_gravity_70s',
                    'free_air_gravity_100s',
                    'free_air_gravity_140s',
                ],
                sep=r"\s+",
            )
            df = df.dropna()
            df = utils.points_inside_region(df, getz_region)
            dfs.append(df)

        df = pd.concat(dfs)
        df.to_csv(f"{pooch.os_cache('pooch')}/goat/gravity/oib_getz.csv", index=False)

    if count>0:
        logger.warning(f"Failed to download {count} files")

    return df.rename(columns={"x": "easting", "y": "northing"})

In [9]:
fetch_oib_gravity()

txt files:   0%|          | 0/42 [00:00<?, ?it/s]


NameError: name 'utils' is not defined